Tarefa 63 - Identificación de caras con DLIB

Fai un programa de consola, que pasándolle unha imaxe coa cara de unha única persoa e o seu nome engada o vector que identifica a cara a un ficheiro (CSV, SQiIte, etc), onde quede asociado o nome co vector.

Este sistema de almacenaxe debería ser válido para posteriormente utilizalo noutros programas.

In [1]:
import face_recognition
import csv

def extract_face_features(image_path):
    # Cargar la imagen
    image = face_recognition.load_image_file(image_path)

    # Detectar caras en la imagen
    face_locations = face_recognition.face_locations(image)

    # Verificar si se detectó exactamente una cara
    if len(face_locations) != 1:
        print("Se requiere exactamente una cara en la imagen.")
        return None, None

    # Extraer el vector de características de la cara
    face_encoding = face_recognition.face_encodings(image, known_face_locations=face_locations)[0]

    return face_encoding

def save_face_features_to_csv(image_path, name, csv_filename):
    # Extraer el vector de características de la cara
    face_encoding = extract_face_features(image_path)

    # Verificar si se detectó una cara
    if face_encoding is None:
        return

    # Abrir el archivo CSV en modo de escritura
    with open(csv_filename, 'a', newline='') as csvfile:
        # Crear un objeto escritor CSV
        csv_writer = csv.writer(csvfile)

        # Escribir el nombre y el vector de características en el archivo CSV
        csv_writer.writerow([name] + face_encoding.tolist())

    print("Vector de características de la cara guardado en el archivo CSV.")

if __name__ == "__main__":
    # Ruta de la imagen con la cara de la persona
    image_path = "chicote.jpg"

    # Nombre de la persona
    name = "Chicote"

    # Nombre del archivo CSV para almacenar los vectores de características de la cara
    csv_filename = "caras.csv"

    # Llamar a la función para guardar el vector de características de la cara en el archivo CSV
    save_face_features_to_csv(image_path, name, csv_filename)


FileNotFoundError: [Errno 2] No such file or directory: 'chicote.jpg'

In [16]:
import cv2
import csv

# Cargamos el clasificador preentrenado para detección de caras
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Lista para almacenar las coordenadas de las caras detectadas
coordenadas_caras = []

# Función para procesar una imagen y detectar caras
def procesar_imagen(nombre_imagen):
    # Cargar la imagen
    imagen = cv2.imread(nombre_imagen)
    # Convertir la imagen a escala de grises
    gray = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
    # Detectar caras en la imagen
    caras = face_cascade.detectMultiScale(gray, 1.1, 4)
    # Guardar las coordenadas de las caras detectadas
    for (x, y, w, h) in caras:
        coordenadas_caras.append((x, y, x + w, y + h))  # Guardamos las coordenadas de la esquina superior izquierda y la esquina inferior derecha

# Procesar cada imagen y grabar las coordenadas de las caras detectadas en el archivo CSV
for imagen in ["chicote.jpg", "images.jpeg", "Imaxe.jpg", "isaac.jpg", "julie.jpeg", "lily.jpg", "manu.jpeg", "phill.png", "todos.png"]:
    procesar_imagen(imagen)

# Escribir las coordenadas en el archivo CSV
with open('caras.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['x1', 'y1', 'x2', 'y2'])  # Escribir la cabecera del archivo CSV
    for coordenada in coordenadas_caras:
        writer.writerow(coordenada)

print("Las coordenadas de las caras han sido guardadas en el archivo 'caras.csv'.")


Las coordenadas de las caras han sido guardadas en el archivo 'caras.csv'.


In [18]:
import face_recognition
import csv

def extract_face_features(image_path):
    # Cargar la imagen
    image = face_recognition.load_image_file(image_path)

    # Detectar caras en la imagen
    face_locations = face_recognition.face_locations(image)

    # Verificar si se detectaron caras en la imagen
    if not face_locations:
        print("No se detectaron caras en la imagen:", image_path)
        return []

    # Inicializar una lista para almacenar los vectores de características de las caras
    face_encodings = []

    # Iterar sobre las ubicaciones de las caras y extraer los vectores de características
    for face_location in face_locations:
        top, right, bottom, left = face_location
        face_image = image[top:bottom, left:right]
        face_encoding = face_recognition.face_encodings(face_image)
        if face_encoding:
            face_encodings.append(face_encoding[0])

    return face_encodings

def save_face_features_to_csv(image_folder, csv_filename):
    # Obtener una lista de todos los archivos de imagen en la carpeta
    import os
    image_files = [f for f in os.listdir(image_folder) if os.path.isfile(os.path.join(image_folder, f))]

    # Abrir el archivo CSV en modo de escritura
    with open(csv_filename, 'w', newline='') as csvfile:
        # Crear un objeto escritor CSV
        csv_writer = csv.writer(csvfile)

        # Iterar sobre los archivos de imagen y guardar los vectores de características en el archivo CSV
        for image_file in image_files:
            image_path = os.path.join(image_folder, image_file)
            face_encodings = extract_face_features(image_path)
            if face_encodings:
                for face_encoding in face_encodings:
                    csv_writer.writerow([image_file] + face_encoding.tolist())

    print("Vectores de características de las caras guardados en el archivo CSV.")

if __name__ == "__main__":
    # Carpeta que contiene las imágenes de las caras
    image_folder = "caras/"

    # Nombre del archivo CSV para almacenar los vectores de características de las caras
    csv_filename = "caras.csv"

    # Llamar a la función para guardar los vectores de características de las caras en el archivo CSV
    save_face_features_to_csv(image_folder, csv_filename)


FileNotFoundError: [Errno 2] No such file or directory: 'caras/'

In [1]:
import dlib
import numpy as np
import csv

def extract_face_vector(image_path):
    # Cargar el detector de caras de DLIB
    detector = dlib.get_frontal_face_detector()
    # Cargar el modelo pre-entrenado de detección de puntos faciales de DLIB
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    # Cargar el modelo pre-entrenado de reconocimiento facial de DLIB
    facerec = dlib.face_recognition_model_v1("dlib_face_recognition_resnet_model_v1.dat")
    
    # Cargar la imagen
    img = dlib.load_rgb_image(image_path)
    
    # Detectar caras en la imagen
    dets = detector(img, 1)
    
    if len(dets) != 1:
        print("Error: Se esperaba una única cara en la imagen.")
        return None, None
    
    # Detectar puntos faciales
    shape = predictor(img, dets[0])
    # Calcular el vector de características de la cara
    face_descriptor = facerec.compute_face_descriptor(img, shape)
    
    return face_descriptor

def add_face_to_database(image_path, name, database_file):
    # Extraer el vector de características de la cara
    face_vector = extract_face_vector(image_path)
    
    if face_vector is not None:
        # Abrir o crear el archivo CSV de la base de datos
        with open(database_file, mode='a', newline='') as file:
            writer = csv.writer(file)
            # Escribir el nombre y el vector de la cara en el archivo CSV
            writer.writerow([name] + list(face_vector))
        print("Cara añadida a la base de datos correctamente.")
    else:
        print("No se pudo extraer la cara de la imagen.")

# Ruta de la imagen con la cara de la persona
image_path = '/caras/chicote.jpg'  # Reemplaza 'cara_persona.jpg' con la ruta de tu imagen
# Nombre de la persona
name = 'Persona'

# Archivo CSV donde se almacenará la base de datos de caras
database_file = 'database.csv'

# Llamar a la función para agregar la cara a la base de datos
add_face_to_database(image_path, name, database_file)


RuntimeError: Unable to open dlib_face_recognition_resnet_model_v1.dat for reading.